In [71]:
import pandas as pd
import numpy as np
from sqlalchemy import create_engine, text
import os
from dotenv import load_dotenv
import warnings
warnings.filterwarnings("ignore")

# Load data SQL

In [72]:
def load_sql(query: str) -> pd.DataFrame:
    load_dotenv()
    db_user= os.getenv("DB_USER")
    db_password= os.getenv("DB_PASSWORD")
    db_host= os.getenv("DB_HOST")
    db_name= os.getenv("DB_NAME")
    engine = create_engine(f'postgresql://{db_user}:{db_password}@{db_host}/{db_name}')
    with engine.connect() as conn:
        df = pd.read_sql_query(text(query), conn)
        return df

pd.set_option('display.float_format', lambda x: '%.2f' % x)
df = load_sql("SELECT * FROM raw.pharmacy_sales;")
df = df.sort_values(by=["distributor", "product_name", "year", "city", "month"])
df.head()

,distributor,customer_name,city,country,latitude,longitude,channel,sub_channel,product_name,product_class,quantity,price,sales,month,year,sales_rep_name,manager,sales_team
207687,Bashirian-Kassulke,Rogahn-Klein Pharma Plc,Leinfelden-Echterdingen,Germany,48.69,9.14,Pharmacy,Institution,Abatatriptan,Antibiotics,2.00,742.00,1484.00,February,2020,Stella Given,Alisha Cordwell,Charlie
187350,Bashirian-Kassulke,Runolfsson-Halvorson Pharm,Rheinberg,Germany,51.55,6.60,Pharmacy,Retail,Abranatal Lysoprosate,Antiseptics,15826.00,681.00,10777506.00,August,2019,Mary Gerrard,Britanny Bold,Delta
254078,Bashirian-Kassulke,Hane Ltd Pharmaceutical Ltd,Aichach,Germany,48.45,11.13,Hospital,Private,Abranatal Lysoprosate,Antiseptics,432.00,681.00,294192.00,December,2020,Anne Wu,Britanny Bold,Delta
175417,Bashirian-Kassulke,Doyle-Tillman Pharmaceutical Limited,Zirndorf,Germany,49.45,10.95,Pharmacy,Institution,Acantaine,Antibiotics,50.00,66.00,3300.00,June,2019,Thompson Crawford,James Goodwill,Alfa
246485,Bashirian-Kassulke,"Langworth, Olson and Satterfield Pharmacy",Meschede,Germany,51.35,8.28,Hospital,Government,Aciprex,Antipiretics,150.00,421.00,63150.00,November,2020,Thompson Crawford,James Goodwill,Alfa


In [73]:
df[['quantity', 'sales']].head(10)

,quantity,sales
207687,2.00,1484.00
187350,15826.00,10777506.00
254078,432.00,294192.00
175417,50.00,3300.00
246485,150.00,63150.00
232401,20.00,8420.00
53000,2500.00,1695000.00
254079,320.00,216960.00
158400,60.00,1440.00
188559,2000.00,48000.00


# Feature Engineering

In [74]:
features = (df.groupby(["distributor",
                    "channel",
                    "sub_channel",
                    "city",
                    "product_name",
                    "product_class",
                    "sales_team",
                    "year",
                    "month",]).agg(
                total_quantity=("quantity", "sum"),
                total_sales=("sales", "sum"),
                avg_price=("price", "mean"),
                    ).reset_index())

sort_columns = ["distributor", "channel", "sub_channel", "city", 
    "product_name", "product_class", "sales_team", 
    "year", "month"]
features = features.sort_values(by=sort_columns)

In [75]:
grp_cols = [
    "distributor" # Concerned columns to identify what makes features works
]

# Preprocessing total sales
sales_upper_bound = features["total_sales"].quantile(0.90) # Quantile concerned to limit outliers
features["total_sales_clean"] = features["total_sales"].clip(lower=0, upper=sales_upper_bound)

# time series not included
grp = features.groupby(grp_cols)

# add rolling features
features["rolling_avg_sales_3m"] = grp["total_sales_clean"].transform(
    lambda x: x.rolling(window=3, min_periods=1).mean()
)
# Count percentage growth
features["sales_growth_pct"] = grp["total_sales_clean"].transform(
    lambda x: x.pct_change() * 100
)

In [76]:
# Count IQR
Q1 = features["sales_growth_pct"].quantile(0.25)
Q3 = features["sales_growth_pct"].quantile(0.75)
IQR = Q3 - Q1

lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

# -----------------------------------------------------------------------------------------------

growth_upper = features["sales_growth_pct"].quantile(0.90) # Quantile concerned to limit outliers

# Apply clipping
features["sales_growth_pct_clipped"] = features["sales_growth_pct"].clip(
    lower=-100,
    upper=growth_upper
)

In [77]:
# Clean Nan and Inf values
features = features.replace([np.inf, -np.inf], np.nan).fillna(0)
features.head(10)

,distributor,channel,sub_channel,city,product_name,product_class,sales_team,year,month,total_quantity,total_sales,avg_price,total_sales_clean,rolling_avg_sales_3m,sales_growth_pct,sales_growth_pct_clipped
0,Bashirian-Kassulke,Hospital,Government,Altenburg,Symbitrim,Analgesics,Bravo,2019,August,29400.00,15758400.00,536.00,76800.00,76800.00,0.00,0.00
1,Bashirian-Kassulke,Hospital,Government,Bad Salzuflen,Adrecetam Barazoxane,Antimalarial,Bravo,2020,December,16.00,384.00,24.00,384.00,38592.00,-99.50,-99.50
2,Bashirian-Kassulke,Hospital,Government,Bad Tölz,Albuterenone,Antimalarial,Bravo,2019,November,2000.00,164000.00,82.00,76800.00,51328.00,19900.00,1884.19
3,Bashirian-Kassulke,Hospital,Government,Bergkamen,Choriogestrel,Antiseptics,Bravo,2020,September,20.00,6940.00,347.00,6940.00,28041.33,-90.96,-90.96
4,Bashirian-Kassulke,Hospital,Government,Böblingen,Feruprazole,Mood Stabilizers,Charlie,2020,December,565.00,64975.00,115.00,64975.00,49571.67,836.24,836.24
5,Bashirian-Kassulke,Hospital,Government,Crailsheim,Symbitrim,Analgesics,Bravo,2019,August,14700.00,7879200.00,536.00,76800.00,49571.67,18.20,18.20
6,Bashirian-Kassulke,Hospital,Government,Dachau,Ketamara Evogel,Antipiretics,Bravo,2020,December,25.00,17600.00,704.00,17600.00,53125.00,-77.08,-77.08
7,Bashirian-Kassulke,Hospital,Government,Derne,Atrabicin Alkerotec,Antiseptics,Alfa,2019,November,100.00,24900.00,249.00,24900.00,39766.67,41.48,41.48
8,Bashirian-Kassulke,Hospital,Government,Dreieich,Effidomide Evofribrate,Antiseptics,Alfa,2019,February,1440.00,881280.00,612.00,76800.00,39766.67,208.43,208.43
9,Bashirian-Kassulke,Hospital,Government,Elmshorn,Lovephilus,Analgesics,Charlie,2019,October,700.00,407400.00,582.00,76800.00,59500.00,0.00,0.00


# Data Encoding

In [78]:
# Encode data
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()

# Encode features and append all columns to features_encoded dictionary
features_encoded = {}
for col in features.columns:
    if features[col].dtype == 'object':
        features_encoded[col] = le.fit_transform(features[col])
    else:
        features_encoded[col] = features[col].values

features_encoded = pd.DataFrame(features_encoded)
features_encoded.head(10)

,distributor,channel,sub_channel,city,product_name,product_class,sales_team,year,month,total_quantity,total_sales,avg_price,total_sales_clean,rolling_avg_sales_3m,sales_growth_pct,sales_growth_pct_clipped
0,0,0,0,10,206,0,1,2019,1,29400.00,15758400.00,536.00,76800.00,76800.00,0.00,0.00
1,0,0,0,35,14,2,1,2020,2,16.00,384.00,24.00,384.00,38592.00,-99.50,-99.50
2,0,0,0,37,27,2,1,2019,9,2000.00,164000.00,82.00,76800.00,51328.00,19900.00,1884.19
3,0,0,0,51,69,4,1,2020,11,20.00,6940.00,347.00,6940.00,28041.33,-90.96,-90.96
4,0,0,0,101,111,5,2,2020,2,565.00,64975.00,115.00,64975.00,49571.67,836.24,836.24
5,0,0,0,122,206,0,1,2019,1,14700.00,7879200.00,536.00,76800.00,49571.67,18.20,18.20
6,0,0,0,127,134,3,1,2020,2,25.00,17600.00,704.00,17600.00,53125.00,-77.08,-77.08
7,0,0,0,135,58,4,0,2019,9,100.00,24900.00,249.00,24900.00,39766.67,41.48,41.48
8,0,0,0,147,99,4,0,2019,3,1440.00,881280.00,612.00,76800.00,39766.67,208.43,208.43
9,0,0,0,160,148,0,2,2019,10,700.00,407400.00,582.00,76800.00,59500.00,0.00,0.00


In [79]:
# Encode df
df_encoded = {}
for col in df.columns:
    if df[col].dtype == 'object':
        df_encoded[col] = le.fit_transform(df[col])
    else:
        df_encoded[col] = df[col].values

df_encoded = pd.DataFrame(df_encoded)
df_encoded.head(10)

,distributor,customer_name,city,country,latitude,longitude,channel,sub_channel,product_name,product_class,quantity,price,sales,month,year,sales_rep_name,manager,sales_team
0,0,551,375,0,48.69,9.14,1,1,0,1,2.00,742.00,1484.00,3,2020,10,0,2
1,0,581,551,0,51.55,6.60,1,3,3,4,15826.00,681.00,10777506.00,1,2019,7,1,3
2,0,242,7,0,48.45,11.13,0,2,3,4,432.00,681.00,294192.00,2,2020,2,1,3
3,0,149,729,0,49.45,10.95,1,1,5,1,50.00,66.00,3300.00,6,2019,12,2,0
4,0,383,423,0,51.35,8.28,0,0,7,3,150.00,421.00,63150.00,9,2020,12,2,0
5,0,522,709,0,53.57,7.78,0,2,7,3,20.00,421.00,8420.00,1,2020,12,2,0
6,0,681,364,0,48.05,10.87,1,1,13,5,2500.00,678.00,1695000.00,7,2017,3,0,2
7,0,252,700,0,53.52,8.13,1,3,13,5,320.00,678.00,216960.00,2,2020,0,3,1
8,0,256,17,0,51.40,8.06,1,1,14,2,60.00,24.00,1440.00,3,2019,3,0,2
9,0,217,507,0,52.38,8.97,1,1,14,2,2000.00,24.00,48000.00,11,2019,11,3,1


# Split data Separately

In [80]:
# Sort values data based on Distributor, year, and month
features_encoded = features_encoded.sort_values(by=["distributor","year", "month"])

def time_split(feature_encoded, split_date=(2018, 9)):
    train = feature_encoded[
        (feature_encoded["year"] < split_date[0]) |
        ((feature_encoded["year"] == split_date[0]) & (feature_encoded["month"] < split_date[1]))
    ]
    test = feature_encoded[
        (feature_encoded["year"] > split_date[0]) |
        ((feature_encoded["year"] == split_date[0]) & (feature_encoded["month"] >= split_date[1]))
    ]
    return train, test

train, test = time_split(features_encoded)

In [81]:
features_encoded

,distributor,channel,sub_channel,city,product_name,product_class,sales_team,year,month,total_quantity,total_sales,avg_price,total_sales_clean,rolling_avg_sales_3m,sales_growth_pct,sales_growth_pct_clipped
48,0,0,2,301,211,1,3,2017,2,1300.00,414700.00,319.00,76800.00,76800.00,0.00,0.00
54,0,0,2,346,57,0,0,2017,2,6800.00,3774000.00,555.00,76800.00,25957.33,24674.19,1884.19
65,0,0,2,598,231,2,1,2017,2,1100.00,775500.00,705.00,76800.00,76800.00,0.00,0.00
115,0,1,1,552,213,1,1,2017,2,6450.00,2773500.00,430.00,76800.00,52002.67,0.00,0.00
139,0,1,3,140,112,2,3,2017,2,500.00,103000.00,206.00,76800.00,76800.00,0.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
253581,28,1,3,690,113,0,3,2018,11,30.00,22350.00,745.00,22350.00,58650.00,-70.90,-70.90
253619,28,1,3,716,69,4,0,2018,11,55.00,19085.00,347.00,19085.00,6732.67,4756.23,1884.19
253637,28,1,3,733,160,0,3,2018,11,5.00,3025.00,605.00,3025.00,26942.33,-96.06,-96.06
253640,28,1,3,733,216,3,0,2018,11,5500.00,4224000.00,768.00,76800.00,64154.67,97.61,97.61


In [82]:
print("Train shape:", train.shape)
print("Test shape:", test.shape)

Train shape: (125076, 16)
Test shape: (128566, 16)


In [83]:
# Feature engineering for build features
def build_features(feature):
    feature = features_encoded.copy()
    feature["rolling_avg_sales_3m"] = (
        feature.groupby("distributor")["total_sales_clean"]
        .rolling(window=3, min_periods=1)
        .mean()
        .shift(1)
        .reset_index(level=0, drop=True)
    )
    return feature.dropna()

train_fe = build_features(train)
test_fe = build_features(test)

# Split data into X and y

In [84]:
# Split data into X and y
final_features = ["avg_price",
                    "month",
                    "year",
                    "distributor",
                    "product_class",
                    "city"
                   # Optional to include time features (Can remove it if not make data advantages)
                  ]
    
X = features[final_features] # Removing feature non concerned in engineering features
y = features["total_sales_clean"]

# Split data into test and train

In [85]:
from sklearn.preprocessing import RobustScaler

# Scale data
scaler = RobustScaler()

X_train = train_fe[final_features]
X_test = test_fe[final_features]

X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

y_train = train_fe["total_sales_clean"]
y_test = test_fe["total_sales_clean"]

print(f"y_train shape: {y_train.shape}, y_test shape: {y_test.shape}")
print(f"X_train_scaled shape: {X_train_scaled.shape}, X_test_scaled shape: {X_test_scaled.shape}")

y_train shape: (253641,), y_test shape: (253641,)
X_train_scaled shape: (253641, 6), X_test_scaled shape: (253641, 6)


In [86]:
# Convert scaled arrays back to DataFrames
X_train_df = pd.DataFrame(X_train_scaled, columns=X_train.columns, index=X_train.index)
X_test_df = pd.DataFrame(X_test_scaled, columns=X_test.columns, index=X_test.index)
X_train_df.head()

,avg_price,month,year,distributor,product_class,city
54,0.30,-0.67,-1.00,-1.40,-1.00,-0.06
65,0.67,-0.67,-1.00,-1.40,-0.33,0.62
115,0.00,-0.67,-1.00,-1.40,-0.67,0.50
139,-0.55,-0.67,-1.00,-1.40,-0.33,-0.61
154,-0.65,-0.67,-1.00,-1.40,-0.33,0.15


# Machine learning models

In [87]:
from sklearn.linear_model import LinearRegression
from xgboost import XGBRegressor
from sklearn.ensemble import RandomForestRegressor
from catboost import CatBoostRegressor

models = {
    "Linear Regression": LinearRegression(),
    "XGBoost Regressor": XGBRegressor(objective='reg:squarederror',
                                      random_state=42),
    "Random Forest Regressor": RandomForestRegressor(n_estimators=300,
                                                     random_state=42,
                                                     max_depth=10,
                                                     min_samples_split=5,
                                                     min_samples_leaf=2,
                                                     n_jobs=-1),
    "CatBoost Regressor": CatBoostRegressor(iterations=1000,
                                            depth=6,
                                            verbose=0, 
                                            random_state=42)
}

from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score, mean_absolute_percentage_error, root_mean_squared_error
def evaluate_model(y_true, y_pred):
    return {
        "MAE": mean_absolute_error(y_true, y_pred),
        "MSE": mean_squared_error(y_true, y_pred),
        "RMSE": root_mean_squared_error(y_true, y_pred),
        "R2 Score": r2_score(y_true, y_pred),
        "MAPE (%)": mean_absolute_percentage_error(y_true, y_pred) * 100
    }

results = {}
for model_name, model in models.items():
    model.fit(X_train_df, y_train)
    y_pred = model.predict(X_test_df)
    results[model_name] = evaluate_model(y_test, y_pred)

results_df = pd.DataFrame(results).T.sort_values(by="MAE")
print(f"Results:\n{results_df}")

Results:
                             MAE          MSE     RMSE  R2 Score  \
XGBoost Regressor       15916.18 477184038.14 21844.54      0.21   
CatBoost Regressor      16069.46 484495370.62 22011.26      0.20   
Random Forest Regressor 16205.29 491062130.74 22159.92      0.19   
Linear Regression       17749.83 558079894.84 23623.71      0.08   

                                       MAPE (%)  
XGBoost Regressor       94507621806123139072.00  
CatBoost Regressor      94972496336578494464.00  
Random Forest Regressor 95094668881006739456.00  
Linear Regression       84772847079881834496.00  
